# <u>**Accessing scripts folder**</u>

In [1]:
import os, sys
my_path ="D:\\10XAcademy\\Prompt_Engineering_Design" 

os.chdir(my_path)
path_to_module = os.path.abspath(os.getcwd()+"\\scripts")
if path_to_module not in sys.path:
    sys.path.append(path_to_module)

# <u>**Fetching data**</u>

In [2]:
from add_to_SQL_DB import db_execute_fetch
news_dataframe = db_execute_fetch(dBName='prompt_engineering', tablename='newsarticles')

10 records fetched from newsarticles table


# <u>**Dataset Exploration**</u>

In [3]:
news_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     10 non-null     int64         
 1   Domain                 10 non-null     object        
 2   Title                  10 non-null     object        
 3   Description            10 non-null     object        
 4   Body                   10 non-null     object        
 5   Link                   10 non-null     object        
 6   Timestamp              10 non-null     datetime64[ns]
 7   Analyst_Average_Score  10 non-null     int64         
 8   Analyst_Rank           10 non-null     int64         
 9   Reference_Final_Score  10 non-null     float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 928.0+ bytes


In [4]:
news_dataframe.head()

,id,Domain,Title,Description,Body,Link,Timestamp,Analyst_Average_Score,Analyst_Rank,Reference_Final_Score
0,31,rassegnastampa.news,Boris Johnson using a taxpayer-funded jet for ...,…often trigger a protest vote that can upset…t...,Boris Johnson using a taxpayer-funded jet for ...,https://rassegnastampa.news/boris-johnson-usin...,2021-09-09 18:17:46,0,4,1.96
1,32,twitter.com,"Stumbled across an interesting case, a woman f...","Stumbled across an interesting case, a woman f...","Stumbled across an interesting case, a woman f...",http://twitter.com/CoruscaKhaya/status/1435585...,2021-09-08 13:02:46,0,4,12.00
2,33,atpe-tchad.info,Marché Résines dans les peintures et revêtemen...,…COVID-19…COVID…COVID…COVID-19 et Post COVID…C...,Le rapport d’étude de marché Résines dans les ...,http://atpe-tchad.info/2021/09/13/marche-resin...,2021-09-13 07:32:46,0,4,0.05
3,34,badbluetech.bitnamiapp.com,"AI drives data analytics surge, study finds",…hate raiders' linked to automated harassment ...,How to drive the funnel through content market...,http://badbluetech.bitnamiapp.com/p.php?sid=21...,2021-09-11 00:17:46,0,4,6.10
4,35,kryptogazette.com,Triacetin Vertrieb Markt 2021: Globale Unterne...,…Abschnitten und Endanwendungen / Organisation...,Global Triacetin Vertrieb-Markt 2021 von Herst...,https://kryptogazette.com/2021/09/08/triacetin...,2021-09-08 12:47:46,0,4,0.13


In [5]:
news_dataframe['Title'][0]

"Boris Johnson using a taxpayer-funded jet for an election campaign fits a long history of taking things he didn't pay for"

In [6]:
news_dataframe.describe()

,id,Analyst_Average_Score,Analyst_Rank,Reference_Final_Score
count,10.00000,10.000000,10.000000,10.000000
mean,35.50000,0.300000,3.400000,4.532000
std,3.02765,0.674949,1.074968,4.834468
min,31.00000,0.000000,1.000000,0.050000
25%,33.25000,0.000000,3.250000,0.505000
50%,35.50000,0.000000,4.000000,2.180000
75%,37.75000,0.000000,4.000000,9.100000
max,40.00000,2.000000,4.000000,12.000000


In [7]:
news_dataframe.select_dtypes(exclude=['int64','float64','datetime64']).describe()

,Domain,Title,Description,Body,Link
count,10,10,10,10,10
unique,9,10,10,10,10
top,eminetra.co.za,Boris Johnson using a taxpayer-funded jet for ...,…often trigger a protest vote that can upset…t...,Boris Johnson using a taxpayer-funded jet for ...,https://rassegnastampa.news/boris-johnson-usin...
freq,2,1,1,1,1


# <u>**WORD EMBEDDING**</u>

In [8]:
from embed_classify import embed_data
news_title_vectors = embed_data((news_dataframe['Title']).to_list())
news_desc_vectors = embed_data((news_dataframe['Description']).to_list())

# <u>**SVM CLASSIFIER TO PERFORM CLUSTERING**</u>

In [9]:
my_path ="D:\\10XAcademy\\Prompt_Engineering_Design" 

os.chdir(my_path)
path_to_module = os.path.abspath(os.getcwd()+"\\models")
if path_to_module not in sys.path:
    sys.path.append(path_to_module)
from svm_classifier import SVMClassifier
classfier = SVMClassifier(news_title_vectors, news_dataframe['Analyst_Average_Score'])
print(classfier.classification_model().predict(news_desc_vectors))

[0 0 0 0 0 0 0 0 0 0]


In [10]:
print(news_dataframe['Analyst_Average_Score'].to_list())

[0, 0, 0, 0, 0, 1, 0, 2, 0, 0]


# <u>**PROMPT BASED CLASSIFICATION**</u>

In [13]:
import classify_api
examples = classify_api.create_examples(news_dataframe.drop(['Analyst_Average_Score','Timestamp'],axis=1).values.tolist(), news_dataframe['Analyst_Average_Score'].values.tolist())
print(examples)
y_pred = news_dataframe.drop('Timestamp',axis=1).apply(func=classify_api.classify_data, args=(examples,)).tolist()

<class 'pandas.core.series.Series'>


CohereError: invalid type, for proper usage, please refer to https://docs.cohere.ai/classify-reference